In [1]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text, inspect, MetaData

import numpy as np
from pathlib import Path
import pandas as pd
from itertools import product
import datetime as dt

import nxviz as nv
import networkx as nx
from nxviz import layouts, plots, lines
from nxviz import nodes, edges, annotate, highlights
from nxviz.plots import despine, rescale, respine, aspect_equal

from nxviz.utils import edge_table, node_table
from nxviz import encodings as aes

import matplotlib
import matplotlib.pyplot as plt

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = ["Arial"]

c:\Users\Higor Monteiro\anaconda3\envs\mywork\Lib\site-packages\nxviz\__init__.py:18: UserWarning: 
nxviz has a new API! Version 0.7.4 onwards, the old class-based API is being
deprecated in favour of a new API focused on advancing a grammar of network
graphics. If your plotting code depends on the old API, please consider
pinning nxviz at version 0.7.4, as the new API will break your old code.

To check out the new API, please head over to the docs at
https://ericmjl.github.io/nxviz/ to learn more. We hope you enjoy using it!

(This deprecation message will go away in version 1.0.)

  warnings.warn(


## **Definitions**

**Database query**

In [2]:
# -- test
def query_metadata(engine):
    inspector = inspect(engine)
    tables = inspector.get_table_names()
    table_dict = { table_name : inspector.get_columns(table_name) for table_name in tables }
    return table_dict


def perform_query(query_str, engine, batchsize=1000):

    schema_data = {
        'rows': [],
        'columns': [],
    }

    query_str = text(query_str)
    with engine.connect() as conn:
        qres = conn.execute(query_str)
        schema_data['columns'] = list(qres.keys())

        while True:
            rows = qres.fetchmany(batchsize)
            if not rows:
                break
            schema_data["rows"] += [ row for row in rows ]
    
    res_df = pd.DataFrame(schema_data['rows'], columns=schema_data['columns'])
    return res_df

**Network definitions - SIH fluxes**

In [ ]:
class BaseFlux:
    def __init__(self, database_engine):
        ''' 
            ...
            
            Args:
            -----
                database_engine:
                    sqlalchemy.engine.base.Engine. SQLAlchemy engine referring to the SIH database.
                    Expected tables from the database are reduced AIHs, rejected AIHs and professional
                    services for each AIH. 
        '''
        self.engine = database_engine

    def create_c2c_networks(self, start_date, final_date):
        pass

    def create_c2h_networks(self, start_date, final_date):
        pass

## **Test Runnings - Networks**

In [3]:
basepath = Path.home().joinpath("Documents", "data")

warehouse_location = basepath.joinpath("opendatasus")
warehouse_name = "SIH_CNES_WAREHOUSE.db"

engine_url = f"sqlite:///{warehouse_location.joinpath(warehouse_name)}"
engine = create_engine(engine_url)

### **Basic interaction with the database**

In [8]:
q = f'''
    SELECT * FROM aih_reduzida LIMIT 5
'''

df = perform_query(q, engine)
df.head(4)

,N_AIH,UF_ZI,ANO_CMPT,MES_CMPT,ESPEC,IDENT,MUNIC_RES,NASC,SEXO,UTI_MES_IN,...,GESTOR_COD,GESTOR_TP,CNES,INFEHOSP,CID_ASSO,CID_MORTE,COMPLEX,FINANC,RACA_COR,FONTE
0,2314100004788,230000,2014,01,07,1,231180,2014-01-17 00:00:00.000000,3,0,...,00000,0,2785900,None,None,None,02,06,99,RDCE1401
1,2314100004854,230000,2014,01,07,1,230945,2011-09-12 00:00:00.000000,3,0,...,00000,0,2785900,None,None,None,02,06,99,RDCE1401
2,2314100004865,230000,2014,01,07,1,230440,2013-05-19 00:00:00.000000,3,0,...,00000,0,2785900,None,None,None,02,06,99,RDCE1401
3,2314100004876,230000,2014,01,07,1,230440,2013-05-30 00:00:00.000000,1,0,...,00000,0,2785900,None,None,None,02,06,99,RDCE1401


In [9]:
q = f'''
    SELECT * FROM servicos_profissionais LIMIT 5
'''

df = perform_query(q, engine)
df.head(4)

,SP_NAIH,SP_GESTOR,SP_AA,SP_MM,SP_CNES,SP_PROCREA,SP_ATOPROF,SP_TP_ATO,SP_QTD_ATO,SP_PTSP,...,SP_M_PAC,SP_COMPLEX,SP_FINANC,SP_CO_FAEC,SP_PF_CBO,SP_CIDPRI,SP_CIDSEC,SP_QT_PROC,SP_U_AIH,FONTE
0,2314100116548,230000,2014,01,6779522,0303060107,0303060107,None,1,0,...,230730,02,06,None,000000,I110,None,1,0,None
1,2314100116548,230000,2014,01,6779522,0303060107,0303060107,None,1,50,...,230730,02,06,None,225125,I110,None,0,1,None
2,2314100116548,230000,2014,01,6779522,0303060107,0301010170,None,6,120,...,230730,02,06,None,225125,I110,None,6,0,None
3,2314100116548,230000,2014,01,6779522,0303060107,0202010180,None,1,0,...,230730,02,06,None,000000,I110,None,1,0,None


In [7]:
q = f'''
    SELECT * FROM aih_rejeitada LIMIT 5
'''

df = perform_query(q, engine)
df.head(4)

,N_AIH,UF_ZI,ANO_CMPT,MES_CMPT,MUNIC_RES,NASC,SEXO,UTI_MES_IN,UTI_MES_AN,UTI_MES_AL,...,MORTE,NACIONAL,NUM_PROC,CAR_INT,TOT_PT_SP,CNES,RACA_COR,ST_SITUAC,ST_BLOQ,ST_MOT_BLO
0,None,230000,2014,01,230440,1944-08-30 00:00:00.000000,3,0,0,0,...,0,010,None,02,0,2785900,99,1,5,00
1,None,230000,2014,01,230440,1944-08-30 00:00:00.000000,3,0,0,0,...,1,010,None,02,0,2785900,99,1,5,00
2,None,230000,2014,01,230440,1963-05-04 00:00:00.000000,1,0,0,0,...,0,010,None,02,0,2785900,99,1,5,00
3,None,230000,2014,01,230440,1945-09-09 00:00:00.000000,3,0,0,0,...,0,010,None,02,0,2785900,99,1,5,00


In [45]:
q = f'''
    SELECT 
        a.*, b.SP_ATOPROF , b.SP_QTD_ATO
    FROM (
        SELECT 
            N_AIH, CNES, MUNIC_RES, MUNIC_MOV, VAL_TOT,
            SUBSTR(DIAG_PRINC,1,3) as DIAG_CATEG 
        FROM aih_reduzida
        WHERE DT_INTER >= '2014-01-01' AND DT_INTER <= '2014-03-01'
    ) a
    LEFT JOIN servicos_profissionais b
    WHERE a.N_AIH = b.SP_NAIH
'''

df = perform_query(q, engine)
df.head(4)

,N_AIH,CNES,MUNIC_RES,MUNIC_MOV,VAL_TOT,DIAG_CATEG,SP_ATOPROF,SP_QTD_ATO
0,2314100118913,6779522,230480,230730,508.34,S82,0408050527,1
1,2314100118913,6779522,230480,230730,508.34,S82,0302040021,1
2,2314100118913,6779522,230480,230730,508.34,S82,0408050527,1
3,2314100118913,6779522,230480,230730,508.34,S82,0408050527,1


In [49]:
df.head(6)

,N_AIH,CNES,MUNIC_RES,MUNIC_MOV,VAL_TOT,DIAG_CATEG,SP_ATOPROF,SP_QTD_ATO
0,2314100118913,6779522,230480,230730,508.34,S82,0408050527,1
1,2314100118913,6779522,230480,230730,508.34,S82,0302040021,1
2,2314100118913,6779522,230480,230730,508.34,S82,0408050527,1
3,2314100118913,6779522,230480,230730,508.34,S82,0408050527,1
4,2314100118913,6779522,230480,230730,508.34,S82,0301010170,2
5,2314100118913,6779522,230480,230730,508.34,S82,0204060125,1


In [51]:
edgelist = df.groupby(["MUNIC_RES", "MUNIC_MOV"])["SP_ATOPROF"].value_counts().reset_index()
edgelist = pd.pivot_table(edgelist, index=["MUNIC_RES", "MUNIC_MOV"], columns="SP_ATOPROF", values="count").fillna(0)
edgelist["SOMA"] = edgelist.apply(sum, axis=1)
edgelist.sort_values(by="SOMA", ascending=False)

,SP_ATOPROF,0201010020,0201010046,0201010062,0201010143,0201010160,0201010194,0201010208,0201010216,0201010224,0201010267,...,0802010067,0802010083,0802010105,0802010121,0802010148,0802010156,0802010199,0802010237,0802020011,SOMA
MUNIC_RES,MUNIC_MOV,,,,,,,,,,,,,,,,,,,,,
230440,230440,0.0,0.0,0.0,2.0,2.0,0.0,15.0,6.0,8.0,7.0,...,1162.0,1850.0,0.0,542.0,0.0,244.0,3383.0,20.0,76.0,237566.0
231290,231290,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,74.0,262.0,0.0,30.0,30.0,0.0,196.0,0.0,8.0,17747.0
230370,230440,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,60.0,90.0,0.0,64.0,0.0,28.0,227.0,0.0,3.0,15485.0
230730,230730,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,62.0,0.0,0.0,64.0,0.0,0.0,97.0,0.0,0.0,15195.0
230420,230420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,142.0,0.0,0.0,0.0,123.0,0.0,0.0,12885.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230540,230380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
230590,231100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
230500,230423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [62]:
def select_period_aih(engine, start_date, final_date, diag_level=0):
    '''
        Filter the AIH records for the period selected and considering the 
        diagnostic level of information required.

        'diag_level' refers to the number of chars to be considered in the 
        diagnostic ICD-10 of the hospital admissions
    '''
    if diag_level>4:
        diag_level = 4
    elif diag_level<0:
        diag_level = 0
    
    query = f'''
        SELECT 
            N_AIH, CNES, MUNIC_RES, MUNIC_MOV, VAL_TOT,
            SUBSTR(DIAG_PRINC,1,{diag_level}) as DIAG_CATEG 
        FROM aih_reduzida
        WHERE DT_INTER >= '{start_date.strftime("%Y-%m-%d")}' AND DT_INTER <= '{final_date.strftime("%Y-%m-%d")}'
    '''
    df = perform_query(query, engine)
    return df

def select_period_aih_services(engine, start_date, final_date):
    '''
    
    '''
    q = f'''
        SELECT 
            a.*, b.SP_ATOPROF , b.SP_QTD_ATO
        FROM (
            SELECT 
                N_AIH, CNES, MUNIC_RES, MUNIC_MOV
            FROM aih_reduzida
            WHERE DT_INTER >= '{start_date.strftime("%Y-%m-%d")}' AND DT_INTER <= '{final_date.strftime("%Y-%m-%d")}'
        ) a
        LEFT JOIN servicos_profissionais b
        WHERE a.N_AIH = b.SP_NAIH
    '''
    df = perform_query(query, engine)
    return df
    

def edgelist_for_c2c(engine, start_date, final_date, diag_level, mode='people'):
    '''
        ...
    '''
    df = select_period_aih(engine, start_date, final_date, diag_level=diag_level)
    if mode == 'people':
        edgelist = df.groupby(["MUNIC_RES", "MUNIC_MOV"])["DIAG_CATEG"].value_counts().reset_index()
        edgelist = pd.pivot_table(edgelist, index=["MUNIC_RES", "MUNIC_MOV"], columns="DIAG_CATEG", values="count").fillna(0)
    elif mode == 'money':
        edgelist = df.groupby(["MUNIC_RES", "MUNIC_MOV", "DIAG_CATEG"])["VAL_TOT"].sum().reset_index()
        edgelist = pd.pivot_table(edgelist, index=["MUNIC_RES", "MUNIC_MOV"], columns="DIAG_CATEG", values="VAL_TOT").fillna(0)
    edgelist["SOMA"] = edgelist.apply(sum, axis=1)
    return edgelist

def edgelist_for_c2h(engine, start_date, final_date, diag_level, mode='people'):
    '''
        ...
    '''
    df = select_period_aih(engine, start_date, final_date, diag_level=diag_level)
    if mode == 'people':
        edgelist = df.groupby(["MUNIC_RES", "CNES"])["DIAG_CATEG"].value_counts().reset_index()
        edgelist = pd.pivot_table(edgelist, index=["MUNIC_RES", "CNES"], columns="DIAG_CATEG", values="count").fillna(0)
    elif mode == 'money':
        edgelist = df.groupby(["MUNIC_RES", "CNES", "DIAG_CATEG"])["VAL_TOT"].sum().reset_index()
        edgelist = pd.pivot_table(edgelist, index=["MUNIC_RES", "CNES"], columns="DIAG_CATEG", values="VAL_TOT").fillna(0)
    edgelist["SOMA"] = edgelist.apply(sum, axis=1)
    return edgelist

def edgelist_services_c2c(engine, start_date, final_date):
    '''
        ...
    '''
    df = select_period_aih_services(engine, start_date, final_date)
    edgelist = df.groupby(["MUNIC_RES", "MUNIC_MOV"])["SP_ATOPROF"].value_counts().reset_index()
    edgelist = pd.pivot_table(edgelist, index=["MUNIC_RES", "MUNIC_MOV"], columns="SP_ATOPROF", values="count").fillna(0)
    edgelist["SOMA"] = edgelist.apply(sum, axis=1)
    return df, edgelist

def edgelist_services_c2h(engine, start_date, final_date):
    '''
        ...
    '''
    df = select_period_aih_services(engine, start_date, final_date)
    edgelist = df.groupby(["MUNIC_RES", "CNES"])["SP_ATOPROF"].value_counts().reset_index()
    edgelist = pd.pivot_table(edgelist, index=["MUNIC_RES", "CNES"], columns="SP_ATOPROF", values="count").fillna(0)
    edgelist["SOMA"] = edgelist.apply(sum, axis=1)
    return df, edgelist


In [17]:
start_date, final_date = dt.datetime(2014, 1, 1), dt.datetime(2014, 12, 31)
#df = select_period_aih(engine, start_date, final_date, diag_level=1)
df = edgelist_for_c2c(engine, start_date, final_date, diag_level=1)
df = edgelist_for_c2c(engine, start_date, final_date, diag_level=1)

In [63]:
df, edgel = edgelist_services_c2c(engine, start_date, final_date)

In [59]:
df.sort_values(by="SOMA")

,SP_ATOPROF,0201010020,0201010046,0201010062,0201010143,0201010160,0201010194,0201010208,0201010216,0201010224,0201010267,...,0802010067,0802010083,0802010105,0802010121,0802010148,0802010156,0802010199,0802010237,0802020011,SOMA
MUNIC_RES,MUNIC_MOV,,,,,,,,,,,,,,,,,,,,,
230360,231290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
231050,231330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
280320,231330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
230100,230625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
261110,231330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230420,230420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,142.0,0.0,0.0,0.0,123.0,0.0,0.0,12885.0
230730,230730,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,62.0,0.0,0.0,64.0,0.0,0.0,97.0,0.0,0.0,15195.0
230370,230440,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,60.0,90.0,0.0,64.0,0.0,28.0,227.0,0.0,3.0,15485.0


In [26]:
df = select_period_aih(engine, start_date, final_date, diag_level=1)

In [27]:
df

,N_AIH,CNES,MUNIC_RES,MUNIC_MOV,VAL_TOT,DIAG_CATEG
0,2314100004788,2785900,231180,230440,1259.92,P
1,2314100004854,2785900,230945,230440,683.87,J
2,2314100004865,2785900,230440,230440,174.42,B
3,2314100004876,2785900,230440,230440,582.42,J
4,2314100004898,2785900,230440,230440,2018.58,J
...,...,...,...,...,...,...
480480,2314107223428,3021114,230520,231290,19337.34,G
480481,2314107224803,3021114,230800,231290,24390.13,I
480482,2314106362546,2415488,230420,230420,17223.18,J
480483,2315100816303,2481286,230440,230440,19401.00,P


In [ ]:
df.groupby(["MUNIC_RES", "MUNIC_MOV"])["DIAG_CATEG"].value_counts().reset_index()

In [34]:
edgelist = df.groupby(["MUNIC_RES", "MUNIC_MOV", "DIAG_CATEG"])["VAL_TOT"].sum().reset_index()
edgelist = pd.pivot_table(edgelist, index=["MUNIC_RES", "MUNIC_MOV"], columns="DIAG_CATEG", values="VAL_TOT").fillna(0)
edgelist["SOMA"] = edgelist.apply(sum, axis=1)

In [37]:
edgelist.sort_values(by="SOMA", ascending=False)[:20]

,DIAG_CATEG,A,B,C,D,E,F,G,H,I,J,...,P,Q,R,S,T,W,X,Y,Z,SOMA
MUNIC_RES,MUNIC_MOV,,,,,,,,,,,,,,,,,,,,,
230440,230440,9051137.06,1892637.23,26158175.56,3587891.17,1504110.77,3540351.92,2649994.50,2728868.46,32071591.75,19857661.44,...,9435159.54,3525027.27,1934971.82,13068557.42,4166719.46,0.00,637.19,1186.27,2123947.96,1.912417e+08
231290,231290,1566551.97,111073.77,520010.75,172142.77,106300.09,191068.06,187512.28,33426.16,3393619.57,1170535.43,...,295726.90,82488.51,261965.43,1335289.61,192105.00,1154.13,47.27,2369.79,90164.06,1.372967e+07
230370,230440,696335.33,105794.92,1633518.71,252466.47,48391.54,193043.36,241395.80,47311.29,2055228.51,1469529.24,...,968826.26,223644.58,121130.25,989201.93,249746.91,0.00,0.00,0.00,175329.84,1.310868e+07
230765,230440,440873.03,95376.82,1467806.04,160526.31,61954.41,164555.57,63638.62,71794.11,1868783.25,509583.48,...,867871.95,251115.79,93338.40,781800.63,237623.15,0.00,0.00,0.00,80027.50,8.692051e+06
230730,230730,339423.74,15703.03,58576.96,101837.40,96907.32,593.44,48352.70,2777.66,503651.51,907985.42,...,902265.42,14714.80,31383.74,982433.75,138499.27,0.00,0.00,637.19,45249.27,7.473034e+06
230190,230190,296087.76,75639.88,384605.81,64241.76,54201.90,1623.52,179364.76,2452.73,1013767.63,1029989.46,...,495384.69,43476.32,115686.58,480184.86,48396.89,0.00,0.00,0.00,26386.54,5.770960e+06
230765,230765,115171.82,51205.07,91547.60,178585.13,45201.51,181.17,71419.08,1211.51,198659.25,483689.80,...,120692.09,14541.48,17384.98,672159.52,93919.79,0.00,0.00,171.47,169292.15,5.320453e+06
230420,230420,239087.66,23806.83,10645.93,158858.13,164476.66,782433.75,13836.72,0.00,431329.43,535278.40,...,63857.79,11835.40,132580.06,294026.13,28877.53,0.00,0.00,0.00,31765.52,5.037226e+06
230100,230440,183057.12,31806.01,408514.70,58555.83,14272.81,28455.02,25152.68,3195.70,528087.29,372662.30,...,264436.44,16168.86,44311.17,285986.78,44747.72,0.00,0.00,0.00,62355.30,3.374230e+06


### **Network tests**

In [ ]:
class BaseFlux:
    def __init__(self, database_engine):
        ''' 
            ...
            
            Args:
            -----
                database_engine:
                    sqlalchemy.engine.base.Engine. SQLAlchemy engine referring to the SIH database.
                    Expected tables from the database are reduced AIHs, rejected AIHs and professional
                    services for each AIH. 
        '''
        self.engine = database_engine


In [10]:
type(engine)

sqlalchemy.engine.base.Engine